In [ ]:
from bigbird.core import flags
from bigbird.core import modeling
from bigbird.core import utils
from bigbird.classifier import run_classifier
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
from tqdm import tqdm
import sys

FLAGS = flags.FLAGS
if not hasattr(FLAGS, "f"): flags.DEFINE_string("f", "", "")
FLAGS(sys.argv)

tf.enable_v2_behavior()

In [ ]:
FLAGS.data_dir = "tfds://imdb_reviews/plain_text"
FLAGS.attention_type = "original_full"
FLAGS.max_encoder_length = 512  # reduce for quicker demo on free colab
FLAGS.learning_rate = 1e-5
FLAGS.num_train_steps = 1000
FLAGS.attention_probs_dropout_prob = 0.0
FLAGS.hidden_dropout_prob = 0.0
FLAGS.use_gradient_checkpointing = True
FLAGS.vocab_model_file = "gpt2"

## Define classification model

In [ ]:
model = modeling.BertModel(bert_config)
headl = run_classifier.ClassifierLossLayer(
        bert_config["hidden_size"], bert_config["num_labels"],
        bert_config["hidden_dropout_prob"],
        utils.create_initializer(bert_config["initializer_range"]),
        name=bert_config["scope"]+"/classifier")

In [ ]:
@tf.function(experimental_compile=True)
def fwd_bwd(features, labels):
  with tf.GradientTape() as g:
    _, pooled_output = model(features, training=True)
    loss, log_probs = headl(pooled_output, labels, True)
    # print("loss:",loss)
  grads = g.gradient(loss, model.trainable_weights+headl.trainable_weights)
  return loss, log_probs, grads

## Dataset pipeline

In [ ]:
train_input_fn = run_classifier.input_fn_builder(
        data_dir=FLAGS.data_dir,
        vocab_model_file=FLAGS.vocab_model_file,
        max_encoder_length=FLAGS.max_encoder_length,
        substitute_newline=FLAGS.substitute_newline,
        is_training=True)
dataset = train_input_fn({'batch_size': 32})

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling imdb_reviews-train.tfrecord...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling imdb_reviews-test.tfrecord...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling imdb_reviews-unsupervised.tfrecord...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


## Train

In [ ]:
import time
start_time = time.time()

In [ ]:
opt = tf.keras.optimizers.Adam(FLAGS.learning_rate)
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

for i, ex in enumerate(tqdm(dataset.take(FLAGS.num_train_steps), position=0)):
  loss, log_probs, grads = fwd_bwd(ex[0], ex[1])
  opt.apply_gradients(zip(grads, model.trainable_weights+headl.trainable_weights))
  train_loss(loss)
  train_accuracy(tf.one_hot(ex[1], 2), log_probs)
  if (i+1)% 50 == 0:
    print('Loss = {}  Accuracy = {}'.format(train_loss.result().numpy(), train_accuracy.result().numpy()))


100%|██████████| 1000/1000 [2:07:24<00:00,  7.64s/it]Loss = 0.5228100419044495  Accuracy = 0.9243752002716061



In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 7944.823274374008 seconds ---


## Eval

In [ ]:
@tf.function(experimental_compile=True)
def fwd_only(features, labels):
  _, pooled_output = model(features, training=False)
  loss, log_probs = headl(pooled_output, labels, False)
  return loss, log_probs

In [ ]:
eval_input_fn = run_classifier.input_fn_builder(
        data_dir=FLAGS.data_dir,
        vocab_model_file=FLAGS.vocab_model_file,
        max_encoder_length=FLAGS.max_encoder_length,
        substitute_newline=FLAGS.substitute_newline,
        is_training=False)
eval_dataset = eval_input_fn({'batch_size': 32})

In [ ]:
eval_loss = tf.keras.metrics.Mean(name='eval_loss')
eval_accuracy = tf.keras.metrics.CategoricalAccuracy(name='eval_accuracy')

for i, ex in enumerate(tqdm(eval_dataset, position=0)):
  loss, log_probs = fwd_only(ex[0], ex[1])
  eval_loss(loss)
  eval_accuracy(tf.one_hot(ex[1], 2), log_probs)
  # if (i+1)% 50 == 0:
  #   print('Loss = {}  Accuracy = {}'.format(eval_loss.result().numpy(), eval_accuracy.result().numpy()))

print('Loss = {}  Accuracy = {}'.format(eval_loss.result().numpy(), eval_accuracy.result().numpy()))

100%|██████████| 781/781 [03:11<00:00,  4.08it/s]Loss = 0.378199964761734  Accuracy = 0.933053507804871


In [ ]:
# eval_input_fn = run_classifier.input_fn_builder(
#         data_dir=FLAGS.data_dir,
#         vocab_model_file=FLAGS.vocab_model_file,
#         max_encoder_length=FLAGS.max_encoder_length,
#         substitute_newline=FLAGS.substitute_newline,
#         is_training=False, )
# eval_dataset = eval_input_fn({'batch_size': 32})

In [ ]:
student_bert_config = flags.as_dictionary()

In [ ]:
student_bert_config['intermediate_size'] = 512
# student_bert_config['hidden_size']= 256
student_bert_config['iterations_per_loop']= '800'
student_bert_config['num_attention_heads']= 8
student_bert_config['num_hidden_layers'] = 3

In [ ]:
student_bert_config

{'attention_probs_dropout_prob': 0.0,
 'attention_type': 'block_sparse',
 'block_size': 16,
 'data_dir': 'tfds://imdb_reviews/plain_text',
 'do_eval': False,
 'do_export': False,
 'do_train': True,
 'eval_batch_size': 8,
 'gcp_project': None,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.0,
 'hidden_size': 768,
 'init_checkpoint': None,
 'initializer_range': 0.02,
 'intermediate_size': 512,
 'iterations_per_loop': '1000',
 'learning_rate': 1e-05,
 'master': None,
 'max_encoder_length': 512,
 'max_position_embeddings': 4096,
 'norm_type': 'postnorm',
 'num_attention_heads': 8,
 'num_hidden_layers': 3,
 'num_labels': 2,
 'num_rand_blocks': 3,
 'num_tpu_cores': 8,
 'num_train_steps': 1000,
 'num_warmup_steps': 1000,
 'optimizer': 'AdamWeightDecay',
 'optimizer_beta1': 0.9,
 'optimizer_beta2': 0.999,
 'optimizer_epsilon': 1e-06,
 'output_dir': '/tmp/bigb',
 'rescale_embedding': False,
 'save_checkpoints_steps': 1000,
 'scope': 'bert',
 'substitute_newline': None,
 'tpu_job_name': None,


In [ ]:
student_model = modeling.BertModel(student_bert_config)
student_headl = run_classifier.ClassifierLossLayer(
        student_bert_config["hidden_size"], student_bert_config["num_labels"],
        student_bert_config["hidden_dropout_prob"],
        utils.create_initializer(student_bert_config["initializer_range"]),
        name=student_bert_config["scope"]+"/classifier")

In [ ]:
import numpy as np
alphas = [0.2, 0.3, 0.4,0.5,0.6,0.7,0.8]
for alpha in alphas:  
  @tf.function(experimental_compile=True)
  def student_fwd_bwd(features, labels):
    with tf.GradientTape() as g:
      _, pooled_output_student = student_model(features, training=True)
      student_loss, log_probs_student = student_headl(pooled_output_student, labels, True)
      # print("log_probs_student: ",log_probs_student)
      # one_hot_labels = tf.one_hot(labels, depth=student_bert_config["num_labels"],dtype=tf.float32)
      # student_loss = -tf.reduce_sum(one_hot_labels * log_probs_student, axis=-1)
      # student_loss = - one_hot_labels * tf.math.log(log_probs_student) - (1 - one_hot_labels) * tf.math.log(1 - log_probs_student)

      _, pooled_output_teacher = model(features, training=False)
      teacher_loss, log_probs_teacher = headl(pooled_output_teacher, labels, False)
      # print("log_probs_teacher: ",log_probs_teacher)
      
      # distil_loss = - log_probs_teacher * tf.math.log(log_probs_student) - (1 - log_probs_teacher) * tf.math.log(1 - log_probs_student)
      
      # mse = tf.keras.losses.MeanSquaredError()
      # distil_loss = mse(log_probs_teacher, log_probs_student)
      # print("distil_loss.numpy(): ",distil_loss.numpy())
      # print("student_loss: ",student_loss.shape)
      # print("distil_loss: ",distil_loss.shape)
      distil_loss = abs(student_loss-teacher_loss)
      loss = alpha * student_loss + (1-alpha) * distil_loss
      # loss = student_loss
      print('\nloss:',loss,'\nstudent_loss:',student_loss)
      # loss = tf.reduce_mean(loss, axis=-1)
      # tf.print(loss)
    grads = g.gradient(loss, student_model.trainable_weights+headl.trainable_weights)
    return loss, log_probs_student, grads

  train_input_fn = run_classifier.input_fn_builder(
          data_dir=FLAGS.data_dir,
          vocab_model_file=FLAGS.vocab_model_file,
          max_encoder_length=FLAGS.max_encoder_length,
          substitute_newline=FLAGS.substitute_newline,
          is_training=True)
  dataset = train_input_fn({'batch_size': 32})

  import time
  start_time = time.time()
  opt = tf.keras.optimizers.Adam(FLAGS.learning_rate)
  train_loss = tf.keras.metrics.Mean(name='train_loss')
  train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

  for i, ex in enumerate(tqdm(dataset.take(FLAGS.num_train_steps), position=0)):
    loss, log_probs, grads = student_fwd_bwd(ex[0], ex[1])
    opt.apply_gradients(zip(grads, student_model.trainable_weights+student_headl.trainable_weights))
    train_loss(loss)
    # print(train_loss(loss))
    train_accuracy(tf.one_hot(ex[1], 2), log_probs)
    if (i+1)% 50 == 0:
      print('Loss = {}  Accuracy = {}'.format(train_loss.result().numpy(), train_accuracy.result().numpy()))

  print("--- %s seconds ---" % (time.time() - start_time))

  @tf.function(experimental_compile=True)
  def fwd_only(features, labels):
    _, pooled_output = student_model(features, training=False)
    loss, log_probs = student_headl(pooled_output, labels, False)
    return loss, log_probs


  eval_input_fn = run_classifier.input_fn_builder(
          data_dir=FLAGS.data_dir,
          vocab_model_file=FLAGS.vocab_model_file,
          max_encoder_length=FLAGS.max_encoder_length,
          substitute_newline=FLAGS.substitute_newline,
          is_training=False)
  eval_dataset = eval_input_fn({'batch_size': 32})

  eval_loss = tf.keras.metrics.Mean(name='eval_loss')
  eval_accuracy = tf.keras.metrics.CategoricalAccuracy(name='eval_accuracy')

  for ex in tqdm(eval_dataset, position=0):
    loss, log_probs = fwd_only(ex[0], ex[1])
    eval_loss(loss)
    eval_accuracy(tf.one_hot(ex[1], 2), log_probs)
  print('Loss = {}  Accuracy = {}'.format(eval_loss.result().numpy(), eval_accuracy.result().numpy()))

  0%|          | 0/1000 [00:00<?, ?it/s]


loss: Tensor("add:0", shape=(), dtype=float32) 
student_loss: Tensor("bert/classifier/Mean:0", shape=(), dtype=float32)


  5%|▌         | 50/1000 [02:53<43:52,  2.77s/it]

Loss = 0.43946707248687744  Accuracy = 0.4987500011920929


 10%|█         | 100/1000 [05:12<41:36,  2.77s/it]

...

 80%|████████  | 800/1000 [37:19<09:08,  2.74s/it]

Loss = 0.1186467707157135  Accuracy = 0.8753125071525574


 85%|████████▌ | 850/1000 [39:36<06:51,  2.75s/it]

Loss = 0.11829531192779541  Accuracy = 0.8766176700592041


 85%|████████▌ | 852/1000 [39:41<06:46,  2.74s/it]

In [ ]:
student_model = modeling.BertModel(student_bert_config)
student_headl = run_classifier.ClassifierLossLayer(
        student_bert_config["hidden_size"], student_bert_config["num_labels"],
        student_bert_config["hidden_dropout_prob"],
        utils.create_initializer(student_bert_config["initializer_range"]),
        name=student_bert_config["scope"]+"/classifier")

import numpy as np
alphas = [0.2, 0.3,0.4,0.5,0.6,0.7,0.8]
for alpha in alphas:  
  @tf.function(experimental_compile=True)
  def student_fwd_bwd(features, labels):
    with tf.GradientTape() as g:
      _, pooled_output_student = student_model(features, training=True)
      loss, log_probs_student = student_headl(pooled_output_student, labels, True)      
    grads = g.gradient(loss, student_model.trainable_weights+student_headl.trainable_weights)
    return loss, log_probs_student, grads

  train_input_fn = run_classifier.input_fn_builder(
          data_dir=FLAGS.data_dir,
          vocab_model_file=FLAGS.vocab_model_file,
          max_encoder_length=FLAGS.max_encoder_length,
          substitute_newline=FLAGS.substitute_newline,
          is_training=True)
  dataset = train_input_fn({'batch_size': 32})

  import time
  start_time = time.time()
  opt = tf.keras.optimizers.Adam(FLAGS.learning_rate)
  train_loss = tf.keras.metrics.Mean(name='train_loss')
  train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

  for i, ex in enumerate(tqdm(dataset.take(FLAGS.num_train_steps), position=0)):
    loss, log_probs, grads = student_fwd_bwd(ex[0], ex[1])
    opt.apply_gradients(zip(grads, student_model.trainable_weights+student_headl.trainable_weights))
    train_loss(loss)
    # print(train_loss(loss))
    train_accuracy(tf.one_hot(ex[1], 2), log_probs)
    if (i+1)% 50 == 0:
      print('Loss = {}  Accuracy = {}'.format(train_loss.result().numpy(), train_accuracy.result().numpy()))

  print("--- %s seconds ---" % (time.time() - start_time))

  @tf.function(experimental_compile=True)
  def fwd_only(features, labels):
    _, pooled_output = student_model(features, training=False)
    loss, log_probs = student_headl(pooled_output, labels, False)
    return loss, log_probs


  eval_input_fn = run_classifier.input_fn_builder(
          data_dir=FLAGS.data_dir,
          vocab_model_file=FLAGS.vocab_model_file,
          max_encoder_length=FLAGS.max_encoder_length,
          substitute_newline=FLAGS.substitute_newline,
          is_training=False)
  eval_dataset = eval_input_fn({'batch_size': 32})

  eval_loss = tf.keras.metrics.Mean(name='eval_loss')
  eval_accuracy = tf.keras.metrics.CategoricalAccuracy(name='eval_accuracy')

  for ex in tqdm(eval_dataset, position=0):
    loss, log_probs = fwd_only(ex[0], ex[1])
    eval_loss(loss)
    eval_accuracy(tf.one_hot(ex[1], 2), log_probs)
  print('Loss = {}  Accuracy = {}'.format(eval_loss.result().numpy(), eval_accuracy.result().numpy()))

In [ ]:
print("student_model params:",student_model.count_params())
print("teacher_model params:",model.count_params())
(model.count_params()/student_model.count_params())*100